# **Logistics Vehicle Maintenance Recommendation Using Classification Model**

1. **Final Project RMT 045 – Group 01**
- Data Analyst : Rivaldi Revin
- Data Scientist : Hafiz Alfariz
- Data Scientist : I Gusti Ngurah Bagus Dwira Palguna
- Data Engineer : Fhad Saleh

2. **Tujuan**
- Melakukan uji coba model yang telah dibuat dengan data baru untuk melihat apakah armada perlu dipelihara atau tidak.

In [2]:
import joblib
import pandas as pd

In [30]:
# Load model & transformers
xgb_search = joblib.load('best_model_classification.pkl')

In [36]:
# New data
df_new = pd.DataFrame({
    'Vehicle_ID': [92001, 92002, 92003],
    'Make_and_Model': ['Ford F-150', 'Ford F-150', 'Ford F-150'],
    'Year_of_Manufacture': [2019, 2016, 2021],
    'Vehicle_Type': ['Truck', 'Truck', 'Truck'],
    'Usage_Hours': [5000, 8000, 3000],
    'Route_Info': ['Urban', 'Highway', 'Rural'],
    'Load_Capacity': [7.5, 10.0, 6.0],
    'Actual_Load': [6.5, 9.0, 5.0],
    'Last_Maintenance_Date': ['2024-01-10', '2024-03-15', '2024-05-01'],
    'Maintenance_Type': ['Oil Change', 'Engine Overhaul', 'Tire Rotation'],
    'Maintenance_Cost': [250.0, 500.0, 150.0],
    'Engine_Temperature': [120.0, 120.0, 120.0],
    'Tire_Pressure': [32.0, 35.0, 30.0],
    'Fuel_Consumption': [10.0, 15.0, 8.0],
    'Battery_Status': [48.0, 50.0, 47.0],
    'Vibration_Levels': [1.2, 3.5, 0.8],
    'Oil_Quality': [85.0, 70.0, 90.0],
    'Brake_Condition': ['Good', 'Good', 'Good'],
    'Failure_History': [0, 1, 0],
    'Anomalies_Detected': [0, 1, 0],
    'Predictive_Score': [0.5, 0.5, 0.5],
    # 'Maintenance_Required': is the target
    'Weather_Conditions': ['Clear', 'Rainy', 'Clear'],
    'Road_Conditions': ['Urban', 'Highway', 'Rural'],
    'Delivery_Times': [48.0, 120.0, 36.0],
    'Downtime_Maintenance': [1.0, 3.0, 0.5],
    'Impact_on_Efficiency': [0.2, 0.4, 0.1]
})

df_new

,Vehicle_ID,Make_and_Model,Year_of_Manufacture,Vehicle_Type,Usage_Hours,Route_Info,Load_Capacity,Actual_Load,Last_Maintenance_Date,Maintenance_Type,...,Oil_Quality,Brake_Condition,Failure_History,Anomalies_Detected,Predictive_Score,Weather_Conditions,Road_Conditions,Delivery_Times,Downtime_Maintenance,Impact_on_Efficiency
0,92001,Ford F-150,2019,Truck,5000,Urban,7.5,6.5,2024-01-10,Oil Change,...,85.0,Good,0,0,0.5,Clear,Urban,48.0,1.0,0.2
1,92002,Ford F-150,2016,Truck,8000,Highway,10.0,9.0,2024-03-15,Engine Overhaul,...,70.0,Good,1,1,0.5,Rainy,Highway,120.0,3.0,0.4
2,92003,Ford F-150,2021,Truck,3000,Rural,6.0,5.0,2024-05-01,Tire Rotation,...,90.0,Good,0,0,0.5,Clear,Rural,36.0,0.5,0.1


In [37]:
# Feature Engineering
leakage_cols = ["Failure_History", "Anomalies_Detected", "Downtime_Maintenance"]
df_new = df_new.drop(columns=leakage_cols, errors="ignore")

# Convert to datetime
df_new['Last_Maintenance_Date'] = pd.to_datetime(df_new['Last_Maintenance_Date'])
df_new['Last_Maintenance_Date'].dtype

# Derivative Feature
current_date = df_new["Last_Maintenance_Date"].max() 
df_new["Days_Since_Last_Maintenance"] = (current_date - df_new["Last_Maintenance_Date"]).dt.days

# Derivative Feature
df_new["Vehicle_Age"] = current_date.year - df_new["Year_of_Manufacture"]

# Drop useless columns
df_new= df_new.drop(columns=["Vehicle_ID", "Last_Maintenance_Date", 'Predictive_Score','Engine_Temperature'])

In [38]:
# Predict new data
y_pred = xgb_search.predict(df_new)
df_new['Maintenance_Required'] = y_pred

# Check probability to be identified as class 1
y_proba = xgb_search.predict_proba(df_new)[:, 1]  
df_new['Maintenance_Required_Prob'] = y_proba

df_new

,Make_and_Model,Year_of_Manufacture,Vehicle_Type,Usage_Hours,Route_Info,Load_Capacity,Actual_Load,Maintenance_Type,Maintenance_Cost,Tire_Pressure,...,Oil_Quality,Brake_Condition,Weather_Conditions,Road_Conditions,Delivery_Times,Impact_on_Efficiency,Days_Since_Last_Maintenance,Vehicle_Age,Maintenance_Required,Maintenance_Required_Prob
0,Ford F-150,2019,Truck,5000,Urban,7.5,6.5,Oil Change,250.0,32.0,...,85.0,Good,Clear,Urban,48.0,0.2,112,5,0,0.163434
1,Ford F-150,2016,Truck,8000,Highway,10.0,9.0,Engine Overhaul,500.0,35.0,...,70.0,Good,Rainy,Highway,120.0,0.4,47,8,1,0.671436
2,Ford F-150,2021,Truck,3000,Rural,6.0,5.0,Tire Rotation,150.0,30.0,...,90.0,Good,Clear,Rural,36.0,0.1,0,3,0,0.235524


In [47]:
print(df_new.groupby('Maintenance_Required')['Vehicle_Age'].mean())
print(df_new.groupby('Maintenance_Required')['Year_of_Manufacture'].mean())
print(df_new.groupby('Maintenance_Required')['Battery_Status'].mean())
print(df_new.groupby('Maintenance_Required')['Tire_Pressure'].mean())
print(df_new.groupby('Maintenance_Required')['Fuel_Consumption'].mean())
print(df_new.groupby('Maintenance_Required')['Oil_Quality'].mean())
print(df_new.groupby('Maintenance_Required')['Usage_Hours'].mean())
print(df_new.groupby('Maintenance_Required')['Vibration_Levels'].mean())

Maintenance_Required
0    4.0
1    8.0
Name: Vehicle_Age, dtype: float64
Maintenance_Required
0    2020.0
1    2016.0
Name: Year_of_Manufacture, dtype: float64
Maintenance_Required
0    47.5
1    50.0
Name: Battery_Status, dtype: float64
Maintenance_Required
0    31.0
1    35.0
Name: Tire_Pressure, dtype: float64
Maintenance_Required
0     9.0
1    15.0
Name: Fuel_Consumption, dtype: float64
Maintenance_Required
0    87.5
1    70.0
Name: Oil_Quality, dtype: float64
Maintenance_Required
0    4000.0
1    8000.0
Name: Usage_Hours, dtype: float64
Maintenance_Required
0    1.0
1    3.5
Name: Vibration_Levels, dtype: float64
